In [1]:
import os
import sys
module_path = "../src"
#os.environ["WANDB_SILENT"] = "true" # Environment Variable to make wandb silent
if module_path not in sys.path:
    sys.path.append(module_path)
# 1. General Libraries
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# 2. DataModule & Class Libraries
from utils.label_encoder import label_encoder_target
from utils.calc_stat import calc_dataset_mean_std
from utils.FeatureExtractor import get_vector_representation,prepare_projection
from dataset.ImageDataModule import ImageDataModule
from dataset.ImageDataset import ImageDataset
from models.EfficientNetClass import EfficientNetClass
from models.ResNetClass import ResNetClass
# 3. Pytorch & Pytorch Lightning Libraries
from pytorch_lightning import Trainer,seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import torch
# 4. Wandb Tracker Experiements
import wandb

In [13]:
DATA_PATH = '../data/experimental_dataset.csv'
BASE_PATH = Path('/mnt')
BASE_IMG_DIR = BASE_PATH
SEED = 42
model_path = '/mnt/artifacts/models/Experimental/experimental_v1.pt'

In [14]:
df = pd.read_csv(DATA_PATH, sep='\t')
le_encoder,dict_label = label_encoder_target(df,target_col='higher_modality')
df.head()

,img,modality,source,img_path,higher_modality,split_set
0,1423-0127-16-6-3-1.jpg,GEL,clef16,subfigure-classification/2016/train/GGEL/1423-...,EXPERIMENTAL,VAL
1,1423-0127-16-6-5-2.jpg,GEL,clef16,subfigure-classification/2016/train/GGEL/1423-...,EXPERIMENTAL,TRAIN
2,1423-0127-16-6-5-3.jpg,GEL,clef16,subfigure-classification/2016/train/GGEL/1423-...,EXPERIMENTAL,TRAIN
3,1423-0127-17-22-2-1.jpg,GEL,clef16,subfigure-classification/2016/train/GGEL/1423-...,EXPERIMENTAL,VAL
4,1423-0127-17-22-2-2.jpg,GEL,clef16,subfigure-classification/2016/train/GGEL/1423-...,EXPERIMENTAL,TRAIN


In [15]:
# 3.1 Load the Resnet Model
model = ResNetClass.load_from_checkpoint(model_path)
print('*'*5,' Resnet Model Parameters ','*'*5)
print(model.hparams)
print('*'*5,' Resnet Model Children ','*'*5)
children_counter = 0
for n,c in model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1

*****  Resnet Model Parameters  *****
"class_weights":   [ 0.51873145 13.84653465]
"fine_tuned_from": whole
"lr":              1e-06
"mean_dataset":    [0.8631029 0.8630955 0.8634109]
"metric_monitor":  val_avg_loss
"mode_scheduler":  min
"name":            resnet18
"num_classes":     2
"pretrained":      True
"std_dataset":     [0.23193271 0.23197891 0.23184325]
*****  Resnet Model Children  *****
Children Counter:  0  Layer Name:  model


In [16]:
prepare_projection(model ,le_encoder,DATA_PATH,BASE_IMG_DIR,SEED,CLASSF ='experimental' ,VERSION = 1)

Feature Vector for training: 



Feature Vector for Validation: 



Feature Vector for Test: 



***** PCA *****
***** UMAP *****
